# True Case

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/true-case](https://github.com/huseinzol05/Malaya/tree/master/example/true-case).
    
</div>

<div class="alert alert-info">

This module trained on both standard and local (included social media) language structures, so it is save to use for both.
    
</div>

In [1]:
import logging

logging.basicConfig(level=logging.INFO)

In [2]:
%%time

import malaya

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


CPU times: user 5.71 s, sys: 1.12 s, total: 6.83 s
Wall time: 7.85 s


### Explanation

Common third party NLP services like Google Speech to Text or PDF to Text will returned unsensitive case and no punctuations or mistake punctuations and cases. So True Case can help you.

1. jom makan di us makanan di sana sedap -> jom makan di US, makanan di sana sedap.
2. kuala lumpur menteri di jabatan perdana menteri datuk seri dr mujahid yusof rawa hari ini mengakhiri lawatan kerja lapan hari ke jordan turki dan bosnia herzegovina lawatan yang bertujuan mengeratkan lagi hubungan dua hala dengan ketiga tiga negara berkenaan -> KUALA LUMPUR - Menteri di Jabatan Perdana Menteri, Datuk Seri Dr Mujahid Yusof Rawa hari ini mengakhiri lawatan kerja lapan hari ke Jordan, Turki dan Bosnia Herzegovina, lawatan yang bertujuan mengeratkan lagi hubungan dua hala dengan ketiga-tiga negara berkenaan.

True case only,

1. Solve mistake / no punctuations.
2. Solve mistake / unsensitive case.
3. Not correcting any grammar.

### List available Transformer model

In [3]:
malaya.true_case.available_transformer()

INFO:malaya.true_case:tested on generated dataset at https://github.com/huseinzol05/malaya/tree/master/session/true-case


,Size (MB),Quantized Size (MB),CER,Suggested length
small,42.70,13.10,0.024601,256.0
base,234.00,63.80,0.014619,256.0
super-tiny-t5,81.80,27.10,0.025468,256.0
super-super-tiny-t5,39.60,12.00,0.025337,256.0
3x-super-tiny-t5,18.30,4.46,0.048737,256.0
3x-super-tiny-t5-4k,5.03,2.99,0.079891,256.0


### Load Transformer model

```python
def transformer(model: str = 'base', quantized: bool = False, **kwargs):
    """
    Load transformer encoder-decoder model to True Case.

    Parameters
    ----------
    model : str, optional (default='base')
        Model architecture supported. Allowed values:

        * ``'small'`` - Transformer SMALL parameters.
        * ``'base'`` - Transformer BASE parameters.
        * ``'super-tiny-t5'`` - T5 SUPER TINY parameters.
        * ``'super-super-tiny-t5'`` - T5 SUPER SUPER TINY parameters.
        * ``'3x-super-tiny-t5'`` - T5 3X SUPER TINY parameters.
        * ``'3x-super-tiny-t5-4k'`` - T5 3X SUPER TINY 4k vocab size parameters.

    quantized : bool, optional (default=False)
        if True, will load 8-bit quantized model.
        Quantized model not necessary faster, totally depends on the machine.

    Returns
    -------
    result: malaya.model.tf.TrueCase class
    """
```

In [7]:
model = malaya.true_case.transformer()

In [3]:
super_super_tiny = malaya.true_case.transformer(model = 'super-super-tiny-t5')

### Load Quantized model

To load 8-bit quantized model, simply pass `quantized = True`, default is `False`.

We can expect slightly accuracy drop from quantized model, and not necessary faster than normal 32-bit float model, totally depends on machine.

In [6]:
quantized_model = malaya.true_case.transformer(quantized = True)

In [4]:
string1 = 'jom makan di us makanan di sana sedap'
string2 = 'kuala lumpur menteri di jabatan perdana menteri datuk seri dr mujahid yusof rawa hari ini mengakhiri lawatan kerja lapan hari ke jordan turki dan bosnia herzegovina lawatan yang bertujuan mengeratkan lagi hubungan dua hala dengan ketiga tiga negara berkenaan'

#### Predict using greedy decoder

```python
def greedy_decoder(self, strings: List[str]):
    """
    True case strings using greedy decoder.
    Example, "saya nak makan di us makanan di sana sedap" -> "Saya nak makan di US, makanan di sana sedap."

    Parameters
    ----------
    strings : List[str]

    Returns
    -------
    result: List[str]
    """
    return self._greedy_decoder(strings)
```

In [7]:
from pprint import pprint

In [12]:
pprint(model.greedy_decoder([string1, string2]))

['Jom makan di US makanan di sana sedap.',
 'KUALA LUMPUR - Menteri di Jabatan Perdana Menteri, Datuk Seri Dr Mujahid '
 'Yusof Rawa hari ini mengakhiri lawatan kerja lapan hari ke Jordan, Turki dan '
 'Bosnia Herzegovina, lawatan yang bertujuan mengeratkan lagi hubungan dua '
 'hala dengan ketiga-tiga negara berkenaan.']


In [8]:
pprint(super_super_tiny.greedy_decoder([string1]))

['Jom makan di US makanan di sana sedap.']


In [13]:
pprint(quantized_model.greedy_decoder([string1, string2]))

['Jom makan di US makanan di sana sedap.',
 'KUALA LUMPUR - Menteri di Jabatan Perdana Menteri, Datuk Seri Dr Mujahid '
 'Yusof Rawa hari ini mengakhiri lawatan kerja lapan hari ke Jordan, Turki dan '
 'Bosnia Herzegovina, lawatan yang bertujuan mengeratkan lagi hubungan dua '
 'hala dengan ketiga-tiga negara berkenaan.']


In [14]:
import random

def random_uppercase(string):
    string = [c.upper() if random.randint(0,1) else c for c in string]
    return ''.join(string)

In [15]:
r = random_uppercase(string2)
r

'KuAlA LUMPUR menTeri di jAbATaN PErDANA MentERI Datuk SERi Dr mujahid YUsof RaWA HARI Ini mEngakhirI LAwaTAN kERJA lApan hARi Ke JOrdaN tURkI dAN bOsnIa heRzEGoVInA lAwaTAN yAng BeRTUjuAN MeNGERAtKaN lAgI HUBUnGAN dua HaLA dENGAn kETiGa TIGA nEgara BerkenAaN'

In [16]:
pprint(model.greedy_decoder([r]))

['KUALA LUMPUR: Menteri di Jabatan Perdana Menteri, Datuk Seri Dr Mujahid '
 'Yusof Rawa hari ini mengakhiri lawatan kerja lapan hari ke Jordan, Turki dan '
 'Bosnia Herzegovina, lawatan yang bertujuan mengeratkan lagi hubungan dua '
 'hala dengan ketiga tiga negara berkenaan.']


In [17]:
pprint(quantized_model.greedy_decoder([r]))

['KUALA LUMPUR: Menteri di Jabatan Perdana Menteri, Datuk Seri Dr Mujahid '
 'Yusof Rawa hari ini mengakhiri lawatan kerja lapan hari ke Jordan, Turki dan '
 'Bosnia Herzegovina, lawatan yang bertujuan mengeratkan lagi hubungan dua '
 'hala dengan ketiga tiga negara berkenaan.']


#### Predict using beam decoder

```python
def beam_decoder(self, strings: List[str]):
    """
    True case strings using beam decoder, beam width size 3, alpha 0.5 .
    Example, "saya nak makan di us makanan di sana sedap" -> "Saya nak makan di US, makanan di sana sedap."

    Parameters
    ----------
    strings : List[str]

    Returns
    -------
    result: List[str]
    """
```

**T5 models not able to use beam_decoder**.

In [18]:
pprint(model.beam_decoder([string1, string2]))

['Jom makan di US makanan di sana sedap.',
 'KUALA LUMPUR - Menteri di Jabatan Perdana Menteri, Datuk Seri Dr Mujahid '
 'Yusof Rawa hari ini mengakhiri lawatan kerja lapan hari ke Jordan, Turki dan '
 'Bosnia Herzegovina, lawatan yang bertujuan mengeratkan lagi hubungan dua '
 'hala dengan ketiga-tiga negara berkenaan.']


In [19]:
pprint(quantized_model.beam_decoder([string1, string2]))

['Jom makan di US makanan di sana sedap.',
 'KUALA LUMPUR - Menteri di Jabatan Perdana Menteri, Datuk Seri Dr Mujahid '
 'Yusof Rawa hari ini mengakhiri lawatan kerja lapan hari ke Jordan, Turki dan '
 'Bosnia Herzegovina, lawatan yang bertujuan mengeratkan lagi hubungan dua '
 'hala dengan ketiga-tiga negara berkenaan.']
